Definition of the decision variables:

Xi = binary variable, 1 = if a fire station is built in city i
                      0 = otherwise

total number of fire starions that are built: 
             
             z = sum Xi
             
LP model


min z = x1 + x2 + x3 + x4 + x5 + x6
st.

constraints 15 minutes within.

        x1 + x2                      >= 1 (city 1 constraint)
        x1 + x2                + x6  >= 1 (city 2 constraint)
                  x3 + x4            >= 1 (city 3 constraint)
                  x3 + x4 + x5       >= 1 (city 4 constraint)
                       x4 + x5  + x6 >= 1 (city 5 constraint)
             x2           + x5  + x6 >= 1 (city 6 constraint)
            
            xi=0 or 1 i=1,2,3,4,5,6
                

In [5]:
import cplex
import pandas as pd

In [6]:
city_data = pd.read_excel('set_covering_input.xlsx', index_col=0)
#set index_col=0 to get rid of the 1st column (city index) and make it an index
city_data

,1,2,3,4,5,6
City/City,,,,,,
1,0,10,20,30,30,20
2,10,0,25,35,20,10
3,20,25,0,15,30,20
4,30,35,15,0,15,25
5,30,20,30,15,0,14
6,20,10,20,25,14,0


convert the dataframe in a vector of list


In [7]:
cities = city_data.values.tolist() 
cities

[[0, 10, 20, 30, 30, 20],
 [10, 0, 25, 35, 20, 10],
 [20, 25, 0, 15, 30, 20],
 [30, 35, 15, 0, 15, 25],
 [30, 20, 30, 15, 0, 14],
 [20, 10, 20, 25, 14, 0]]

the problem use binary variable and must have 1-0 coefficients so is neccesary to create the matrix with these coefficients


In [8]:


city_min = [] 

for i in range(len(cities)): 
    the_list = []
    for j in range(len(cities[0])):
        if cities[i][j] <= 15:
            the_list.append(1)
        else:
            the_list.append(0)
    city_min.append(the_list)

city_min

[[1, 1, 0, 0, 0, 0],
 [1, 1, 0, 0, 0, 1],
 [0, 0, 1, 1, 0, 0],
 [0, 0, 1, 1, 1, 0],
 [0, 0, 0, 1, 1, 1],
 [0, 1, 0, 0, 1, 1]]

we already have all the coefficients and the LP model, so we can create the lists for the other elements of the model


In [9]:

variable_type = []
obj_func = []
senses = []
city_names = []
rhs = []


for i in range(len(cities)):
    variable_type.append("B")
    obj_func.append(1)
    senses.append(['G'])
    city_names.append('City ' + str(i+1))
    rhs.append([1])
    
print(variable_type)
print(obj_func)
print(senses)
print(city_names)
print(rhs) 
 

['B', 'B', 'B', 'B', 'B', 'B']
[1, 1, 1, 1, 1, 1]
[['G'], ['G'], ['G'], ['G'], ['G'], ['G']]
['City 1', 'City 2', 'City 3', 'City 4', 'City 5', 'City 6']
[[1], [1], [1], [1], [1], [1]]


Define the objective function and the decision variables

In [10]:
cpx = cplex.Cplex()
cpx.objective.set_sense(cpx.objective.sense.minimize)
cpx.variables.add(obj = obj_func, names=city_names, types=variable_type)


range(0, 6)

Aggregate the constraints for each city, in this case we use the method of "adding rows"

In [11]:
for i in range(len(cities)): #create a constraint for each city
    cpx.linear_constraints.add(lin_expr=[[city_names, city_min[i]]], rhs=[rhs[i]],
                             senses=senses[i],names=[city_names[i]])
    

Solve the problem

In [12]:
cpx.solve()
optimal_cities = cpx.solution.get_values()
print()
print("Solution status = ", cpx.solution.get_status(), ":", end=' ')
print(cpx.solution.status[cpx.solution.get_status()])
print("Solution value  = ", cpx.solution.get_objective_value())
print("Open Fire stations in: ")
for i in range(len(cities)):
      if optimal_cities[i] > 0:
            print(city_names[i])

Version identifier: 22.1.0.0 | 2022-03-09 | 1a383f8ce
CPXPARAM_Read_DataCheck                          1
Found incumbent of value 6.000000 after 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 6 rows and 6 columns.
All rows and columns eliminated.
Presolve time = 0.00 sec. (0.01 ticks)

Root node processing (before b&c):
  Real time             =    0.00 sec. (0.01 ticks)
Parallel b&c, 8 threads:
  Real time             =    0.00 sec. (0.00 ticks)
  Sync time (average)   =    0.00 sec.
  Wait time (average)   =    0.00 sec.
                          ------------
Total (root+branch&cut) =    0.00 sec. (0.01 ticks)

Solution status =  101 : MIP_optimal
Solution value  =  2.0
Open Fire stations in: 
City 2
City 4
